# Trabalho 01 - Experimento SSVEP offline

Dennis Felipe Urtubia e Pedro Perozin

# Carregamento dos dados, labels e descritor

In [8]:
import numpy as np
import json
import mne
import scipy
import matplotlib


from scipy.signal import stft
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

data = np.load('files/data.npy')
labels = np.load('files/labels.npy')

desc_file = open('files/descriptor.json')
descriptor = json.loads(desc_file.read())
desc_file.close()

print('Estruturas => dados', data.shape, 'labels', labels.shape)
print(labels)

print(descriptor['frequencies'])

Estruturas => dados (125, 257, 1205) labels (125,)
[4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1 5
 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1
 3 4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1
 5 4 3 2 4 1 2 5 3 4 1 3 1 3]
{'5': 6.66, '4': 7.5, '3': 8.57, '2': 10.0, '1': 12.0}


# Criação do EpochsArray

In [9]:
data = data[:,:256,:]

trial_duration = 5
sampling_frequency = data.shape[-1] / trial_duration
montage = mne.channels.make_standard_montage('EGI_256')
ch_names = data.shape[1]
ch_types = 'eeg'

# primeiramente devemos criar o objeto info
info = mne.create_info(montage.ch_names, sampling_frequency, ch_types)

# definindo a montagem do experimento
info.set_montage(montage)

# por fim a criação do EpochsArray
events = np.array([[index, 0, event] for index, event in enumerate(labels)])
# objeto MNE epoch
epoch = mne.EpochsArray(data, info, events)

125 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


# Aplicando filtros espaciais

In [10]:
channels = ['E108', 'E109', 'E116', 'E125', 'E118', 'E117', 'E126', 'E139', 'E127', 'E138', 'E140', 'E150', 'E151']

filtered_epoch = epoch.copy().pick_channels(channels)

for i in range(10):
    filtered_epoch.filter(l_freq = 5.0, h_freq = 14.0)

# CAR
# mne.set_eeg_reference(filtered_epoch, ref_channels=channels)

Setting up band-pass filter from 5 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 399 samples (1.656 sec)

Setting up band-pass filter from 5 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Fi

# Extração de características

O método `psd_multitaper` nos retorna as características já no domínio da frequência, sem precisar passar por o método stft, por exemplo.

In [11]:
X, _ = mne.time_frequency.psd_multitaper(filtered_epoch)
print('Shape dos dados:', X.shape)

    Using multitaper spectrum estimation with 7 DPSS windows
Shape dos dados: (125, 13, 603)


## Finalizando o vetor de características

In [12]:
X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

print('Shape dos dados:', X.shape)

Shape dos dados: (125, 7839)


## Adaptação do vetor de labels

In [13]:
y = np.load('files/labels.npy')
print('Shape original dos labels', y.shape)

# size = int(X.shape[0] / y.shape[0])
# y = np.concatenate([y for i in range(size)])
# print('Shape final dos labels', y.shape)

Shape original dos labels (125,)


# Classificação

### SVM - Support Vector Machine

Foi utilizado o método `GridSearchCV` da biblioteca `sklearn` para testar o classificador com diversos parâmetros.
É disposto apenas o melhor resultado do experimento.

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import (StratifiedShuffleSplit, GridSearchCV)

parameters_linear = {
    'kernel': ['linear'],
    'C':[0.01, 0.1, 1, 10, 100, 1000]
}

parameters_poly = {
    'kernel': ['poly'],
    'C':[0.01, 0.1, 1, 10, 100, 1000],
    'gamma': [10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]
}

sss = StratifiedShuffleSplit(n_splits=30, test_size=0.3)

grid_search_linear = GridSearchCV(SVC(), parameters_linear, n_jobs=-1, cv=sss)
grid_search_poly = GridSearchCV(SVC(), parameters_poly, n_jobs=-1, cv=sss)

grid_search_linear.fit(X, y)
grid_search_poly.fit(X, y)

print('Melhor resultado linear', grid_search_linear.best_score_, grid_search_linear.best_params_)
print('Melhor resultado poly', grid_search_poly.best_score_, grid_search_poly.best_params_)

Melhor resultado linear 0.3368421052631579 {'C': 0.01, 'kernel': 'linear'}
Melhor resultado poly 0.3149122807017543 {'C': 0.01, 'gamma': 10, 'kernel': 'poly'}
